In [72]:
# CODE FOR LOADING SAVED CHATBOT

# import libraries
import random
import json
import pickle
import numpy as np
import nltk
from keras.models import load_model
from nltk.stem import WordNetLemmatizer

# load intents and saved model output files
lemmatizer = WordNetLemmatizer()
intents = json.loads(open("intents.json").read())
words = pickle.load(open('words.pkl', 'rb'))
labels = pickle.load(open('labels.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

# separate input words from input sentence
def clean_up_sentences(sentence):
	sentence_words = nltk.word_tokenize(sentence)
	sentence_words = [lemmatizer.lemmatize(word)
					for word in sentence_words]
	return sentence_words

# determine if input words in saved model words list
def bagw(sentence):
	sentence_words = clean_up_sentences(sentence)
	bag = [0]*len(words)
	for w in sentence_words:
		for i, word in enumerate(words):
			if word == w:
				bag[i] = 1
	return np.array(bag)

# predict label of sentence based on input words
#update: leveraging list comprehensions and dictionaries to work with confidence scores
def predict_class(sentence):
    bow = bagw(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [{'intent': labels[i], 'probability': str(res[i])}
               for i in range(len(res)) if res[i] > ERROR_THRESHOLD]
    results.sort(key=lambda x: x['probability'], reverse=True)
    return results



# print random response from appropriate responses for label
#if predicted intent has a low confidence score, gives a secondary response
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    probability = float(intents_list[0]['probability'])
    list_of_intents = intents_json['intents']
    result = ""
    if probability < 0.2:  # Adjust the threshold as needed
        result = "I'm sorry, but I'm not sure how to help with that. Could you please provide more information?"
    else:
        for i in list_of_intents:
            if i['tag'] == tag:
                result = random.choice(i['responses'])
                break
    return result

# create infinite loop that prompts for input
print("Chatbot live:")

while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    top_intent = ints[0]
    if res.startswith("I'm sorry"):
        print("Bot:", res)
        # Additional code will be added for handling unclear queries
        
    else:
        print("Bot:", res)
        # Additional code will be added in next trimester for handling normal conversation flow
     



Chatbot live:
hello
1/1 [==============================] - 0s 57ms/step
Hey! How can I assist you today?
thank you
1/1 [==============================] - 0s 36ms/step
You're welcome!
recommend
1/1 [==============================] - 0s 20ms/step
Based on your previous transaction you may save upto


KeyboardInterrupt: ignored